#### Task 1.2: Data Preparation

Improve the quality of your data and prepare it by extracting new features interesting for describing the incidents. Some examples of indicators to be computed are:

- How many males are involved in incidents relative to the total number of males for the same city and in the same period?
- How many injured and killed people have been involved relative to the total injured and killed people in the same congressional district in a given period of time?
- Ratio of the number of killed people in the incidents relative to the number of participants in the incident
- Ratio of unharmed people in the incidents relative to the average of unharmed people in the same period

Note that these examples are not mandatory, and teams can define their own indicators. Each indicator must be correlated with a description and, when necessary, its mathematical formulation. The extracted variables will be useful for the clustering analysis in the second project's task. Once the set of indicators is computed, the team should explore the new features for a statistical analysis, including distributions, outliers, visualizations, and correlations.

For task 1.1 see the corresponding Notebook in [Task 1.1 - Data Understanding](Task1_Data_Understanding.ipynb).

For this task we followed the following check structure: [#WIP]()
1. Data aggregation
2. Reduction of dimensionality
3. Data cleaning
4. Discretization
5. Data transformation
6. Principal Component Analysis via Covariance Matrix
8. Data Similarity via Entropy and proximity coefficients


In [1]:
# This will take a while
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import plotly.offline as py

# for dataset visibility
pd.set_option('display.max_columns', None)

# Set a seed for reproducibility
np.random.seed(42)

# Load dataset from data understanding
#df_incident_du = pd.read_csv('../ds/cleaned/incidents_cleaned.csv')
df = pd.read_hdf('../ds/cleaned/incidents_cleaned.h5', key = 'df')
df_incident_du = df.copy()   
df_poverty_du = pd.read_csv('../ds/cleaned/poverty_cleaned.csv')
df_years_state_du = pd.read_csv('../ds/cleaned/year_state_votes_cleaned.csv')


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Check if dataset loaded correctly
df_incident_du.head()
#df_poverty_du.head()
#df_years_state_du.head()

#df_incident_du.info()

,date,state,city_or_county,latitude,longitude,congressional_district,participant_age1,participant_age_group1,participant_gender1,min_age_participants,max_age_participants,n_participants_child,n_participants_teen,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,incident_characteristics1,year
218445,2014-01-01,Arkansas,Pine Bluff,34.2291,-91.9779,4,26,Adult 18+,Male,26,26,0,0,0,0,1,0,1,2,Shot - Wounded/Injured,2014
180520,2014-01-01,California,Bakersfield,35.3229,-119.0380,23,28,Adult 18+,Male,28,28,0,0,0,0,2,0,0,2,Shot - Wounded/Injured,2014
44077,2014-01-01,Iowa,Iowa City,41.6611,-91.5302,2,35,Adult 18+,Female,35,35,0,0,2,0,1,0,1,2,other,2014
89936,2014-01-01,New York,Brooklyn,40.6505,-73.9573,9,28,Adult 18+,Male,28,30,0,0,1,0,3,0,0,3,Shot - Wounded/Injured,2014
186445,2014-01-01,Colorado,Grand Junction,39.0496,-108.5250,3,47,Adult 18+,Male,47,47,0,0,0,1,0,0,0,1,"Shot - Dead (murder, accidental, suicide)",2014


### Fix little problem with reimporting datasets

CSV files do not store informations about categories and datatypes like the one we saved.

# Joining the datasets

We have three dataset: the one with the incidents, one with poverty by state (just two columns) and one with year, state and how much votes each party got. We want to join the three datasets in order to have a single dataset with all the information we need. 

In [3]:
# We need to do merge function from pandas

df_merged_incident_poverty = pd.merge(df_incident_du ,df_poverty_du, on=['year', 'state'], how='left')

df_merged_incident_poverty.head()

,date,state,city_or_county,latitude,longitude,congressional_district,participant_age1,participant_age_group1,participant_gender1,min_age_participants,max_age_participants,n_participants_child,n_participants_teen,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,incident_characteristics1,year,povertyPercentage
0,2014-01-01,Arkansas,Pine Bluff,34.2291,-91.9779,4,26,Adult 18+,Male,26,26,0,0,0,0,1,0,1,2,Shot - Wounded/Injured,2014,16.8
1,2014-01-01,California,Bakersfield,35.3229,-119.0380,23,28,Adult 18+,Male,28,28,0,0,0,0,2,0,0,2,Shot - Wounded/Injured,2014,14.5
2,2014-01-01,Iowa,Iowa City,41.6611,-91.5302,2,35,Adult 18+,Female,35,35,0,0,2,0,1,0,1,2,other,2014,10.1
3,2014-01-01,New York,Brooklyn,40.6505,-73.9573,9,28,Adult 18+,Male,28,30,0,0,1,0,3,0,0,3,Shot - Wounded/Injured,2014,13.4
4,2014-01-01,Colorado,Grand Junction,39.0496,-108.5250,3,47,Adult 18+,Male,47,47,0,0,0,1,0,0,0,1,"Shot - Dead (murder, accidental, suicide)",2014,10.2



# Idea: Let's try to get number of incidents every 3 months
## Maybe this in data preparation
Since 2018 is the year with less records we could try to visualize what period contains more incidents, let's try to monitor every 3 months: